In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels
import statsmodels.api as sm
import statsmodels.stats.api as sms
import statsmodels.formula.api as smf
import plotly
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import docx
import datetime as dt
import pickle
import random
import math
import time
import os
import re
import colorsys
import arrow
import locale
import requests
import collections
import numbers
import decimal

from warnings import simplefilter

from scipy import stats
from scipy.interpolate import make_interp_spline, BSpline
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from matplotlib.lines import Line2D
import matplotlib.dates as mdates

In [2]:
path_current = os.getcwd()
path_main = os.path.sep.join(path_current.split(os.path.sep)[:-2])

In [3]:
path_current

'/Users/achasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/Мой диск/Data Science Projects/# economy-rus-dash/notebooks/data-import'

In [4]:
path_main

'/Users/achasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/Мой диск/Data Science Projects/# economy-rus-dash'

### To main directory

In [5]:
# change directory to main path
os.chdir(path_main)

### Load libraries, functions, palette, theme

In [6]:
# load theme
%run __theme.ipynb

In [7]:
# load functions
%run __functions.ipynb

In [8]:
# load regions data
path_regions_data = path_main + '/files'
path_change(path_regions_data)

%run regions_data.ipynb

In [9]:
# start stopwatch
start = stopwatch_start()

### Variables

In [10]:
path_files = path_main + '/files'

In [11]:
path_excel_csv = path_main + '/excel-csv'

### Load Files

In [12]:
economics_data = loadit('economics_data', path=path_files, create_empty_dict=True)

### Back to notebook directory

In [13]:
# back to current path
os.chdir(path_current)

# <font color='#2C8B6D'>I. Население</font>

# Section III. Уровень жизни

## 7. Доходы, расходы и условия проживания домашних хозяйств

### 7.1. Доходы и расходы домашних хозяйств на потребление, уровень и структура располагаемых ресурсов домашних хозяйств различных социально-экономических категорий

*Источник: https://rosstat.gov.ru/folder/13397*  
*Единицы измерения: рубли*

*Примечания:*  

In [14]:
income_link = 'https://rosstat.gov.ru/storage/mediabank/Doh07_01.xlsx'

In [15]:
income_raw = pd.read_excel(income_link)

In [16]:
income = income_raw[7:56].copy()

In [17]:
# remove symbols in 'year' row
income.iloc[0] = [to_string(i)[:4] for i in income.iloc[0]]

/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83019/2330403369.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2008' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  income.iloc[0] = [to_string(i)[:4] for i in income.iloc[0]]
/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83019/2330403369.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2009' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  income.iloc[0] = [to_string(i)[:4] for i in income.iloc[0]]
/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83019/2330403369.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2010' has dtype incompatible with float64, please exp

In [18]:
# rename column
income.iloc[0, 0] = 'Тип'

In [19]:
# make header from 0 row
income = transform_make_header_from_rows(
    income, rows_index=7)

In [20]:
# remove spaces in 0 column
income.iloc[:, 0] = [str(i).strip() for i in income.iloc[:, 0]]

In [21]:
# replace
income.iloc[:, 0] = \
    [i.replace('Располагаемые ресурсы - всего', 'Всего') for i in income.iloc[:, 0]]

In [22]:
for column in income.columns:
    income[column] = income[column].apply(lambda x: to_float(x))

In [23]:
income = to_round(income, 1)

In [24]:
income['index'] = income['2003'].copy()

In [25]:
# to NaN values if length less than 10
income['index'] = \
    [i if len(str(i)) > 10 else np.NaN for i in income['index'].values.ravel()]

In [26]:
income = income.reset_index(drop=True)

In [27]:
# concat strings in 'index' and drop unuseful values
income, drop_indexes1 = transform_concat_rows_strings(income, 'index')

In [28]:
# fillna by previous notna
income = transform_fill_values_by_previous(
    data=income, kind='column', column_name='index')

In [29]:
income = income.drop(drop_indexes1, axis=0)

In [30]:
income = income.loc[income['Тип'] != 'в том числе:']

In [31]:
type_values1 = [
    'Всего', 'Денежные расходы', 'Стоимость натуральных поступлений продуктов питания',
    'Стоимость натуральных поступлений непродовольственных товаров и услуг',
    'Сумма сделанных сбережений'
]

In [32]:
j = 0
for index in income.index:
    if not pd.isna(income.loc[index, '2003']):
        new_value = type_values1[j]
        income.loc[index, 'Тип'] = new_value
        j += 1
        if j > 4:
            j = 0

In [33]:
income = income.dropna()

In [34]:
income = income.set_index('index', drop=True)
income.index.name = None

In [35]:
# convert column names to int if possible (isdigit)
income.columns = [int(i) if i.isdigit() else i for i in income.columns]
income.columns.name = None

In [36]:
income = income.map(to_float)

In [37]:
income.head()

,Тип,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Все домашние хозяйства,Всего,3586.6,4446.9,5745.8,7051.6,9156.9,11955.2,12419.4,14633.2,16597.1,...,21198.5,22890.1,23084.8,24209.8,24926.5,26917.7,28995.2,29204.5,31791.2,35066.1
Все домашние хозяйства,Денежные расходы,3148.5,3924.8,5057.9,6199.5,8062.4,10335.3,10737.4,12688.0,14432.0,...,18089.9,19447.2,19293.3,20583.5,21220.9,22926.4,24673.6,24504.8,26226.7,29185.2
Все домашние хозяйства,Стоимость натуральных поступлений продуктов пи...,224.2,225.6,256.0,271.5,301.0,348.3,378.9,422.3,447.1,...,468.6,490.2,585.2,598.8,585.0,569.6,605.0,628.0,656.3,712.8
Все домашние хозяйства,Стоимость натуральных поступлений непродовольс...,66.5,70.1,41.5,49.2,66.0,87.9,94.6,104.3,138.5,...,148.7,149.5,208.9,167.8,177.2,177.9,207.1,191.7,221.8,288.2
Все домашние хозяйства,Сумма сделанных сбережений,147.4,226.4,390.4,531.4,727.5,1183.7,1208.4,1418.5,1579.5,...,2491.2,2803.2,2997.4,2859.6,2943.3,3232.9,3509.5,3880.0,4686.4,4880.0


In [38]:
income[income['Тип'] == 'Всего'].T[1:].iloc[:, -2:].head()

,"Домашние хозяйства, проживающиев городской местности","Домашние хозяйства, проживающиев сельской местности"
2003,4009.0,2423.2
2004,5016.0,2851.3
2005,6529.5,3604.7
2006,7984.9,4495.4
2007,10354.6,5871.1


##### <font color='#AF4035'> Export Dataset to Excel

In [39]:
saveit_excel(
    data=income,
    filename='Домохозяйства',
    path=path_excel_csv,
    sheet='Структура ресурсов'
)

'Структура ресурсов' sheet created in file 'Домохозяйства.xlsx'


### 7.2. Доходы и расходы домашних хозяйств на потребление, уровень и структура расходов на потребление домашних хозяйств различных социально-экономических категорий

*Источник: https://rosstat.gov.ru/folder/13397*  
*Единицы измерения: рубли в среднем на члена семьи в месяц*

*Примечания:*  

In [40]:
consumption_link = 'https://rosstat.gov.ru/storage/mediabank/Doh07_05.xlsx'

In [41]:
consumption_raw = pd.read_excel(consumption_link)

In [42]:
consumption_raw.head(10)

,Обновлено 17.07.2023,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,Доходы и расходы домашних хозяйств на потребление,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Уровень и структура расходов напотребление дом...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,СОСТАВ РАСХОДОВ НА ПОТРЕБЛЕНИЕ ДОМАШНИХ ХОЗЯЙСТВ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,РАЗЛИЧНЫХ СОЦИАЛЬНО-ЭКОНОМИЧЕСКИХ КАТЕГОРИЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,(в среднем на члена домашнего хозяйства в меся...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,2003,2004,2005,2006,2007,2008,2009,2010.0,2011.0,...,2013.000000,2014.000000,2015.0,2016.0,2017.0,2018.0,2019.000,2020.00,2021.000,2022*
8,NaN,Все домашние хозяйства,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Расходы на потребление - всего,2989.3,3582.9,4490,5353.1,6842.6,8561.8,9054,10513.4,11715.1,...,14153.774977,15094.284337,15295.4,16632.5,17319.9,18031.4,19727.701,19180.79,21154.264,23787.9


In [43]:
consumption = consumption_raw[7:55].copy()

In [44]:
# remove symbols in 'year' row
consumption.iloc[0] = [to_string(i)[:4] for i in consumption.iloc[0]]

/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83019/2992359583.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2010' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  consumption.iloc[0] = [to_string(i)[:4] for i in consumption.iloc[0]]
/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83019/2992359583.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2011' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  consumption.iloc[0] = [to_string(i)[:4] for i in consumption.iloc[0]]
/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83019/2992359583.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2012' has dtype incompatible with

In [45]:
# rename column
consumption.iloc[0, 0] = 'Тип'

In [46]:
# make header from 0 row
consumption = transform_make_header_from_rows(
    consumption, rows_index=7)

In [47]:
consumption.iloc[:, 0] = [str(i).strip() for i in consumption.iloc[:, 0]]

In [48]:
consumption.iloc[:, 0] = \
    [i.replace('Расходы на потребление - всего', 'Всего') for i in consumption.iloc[:, 0]]

In [49]:
for column in consumption.columns:
    consumption[column] = consumption[column].apply(
        lambda x: to_float(x))

In [50]:
consumption = to_round(consumption, 1)

In [51]:
consumption['index'] = consumption['2003'].copy()

In [52]:
# to NaN values if length less than 10
consumption_index_values = consumption['index'].values.ravel()
consumption['index'] = \
    [i if len(str(i)) > 10 else np.NaN for i in consumption_index_values]

In [53]:
consumption = consumption.reset_index(drop=True)

In [54]:
# concat strings in 'index' and drop unuseful values
consumption, drop_indexes2 = transform_concat_rows_strings(consumption, 'index')

In [55]:
# fillna by previous notna
consumption = transform_fill_values_by_previous(
    data=consumption, kind='column', column_name='index')

In [56]:
consumption = consumption.drop(drop_indexes2, axis=0)

In [57]:
consumption = consumption.loc[consumption['Тип'] != 'в том числе:']

In [58]:
type_values2 = [
    'Всего',
    'Расходы на питание',
    'Расходы на непродовольственные товары',
    'Расходы на алкогольные напитки',
    'Расходы на оплату услуг',
    'Стоимость услуг, оказанных работодателем бесплатно или по льготным ценам'
]

In [59]:
j = 0
for index in consumption.index:
    if not pd.isna(consumption.loc[index, '2003']):
        new_value = type_values2[j]
        consumption.loc[index, 'Тип'] = new_value
        j += 1
        if j > 5:
            j = 0

In [60]:
consumption = consumption.dropna()

In [61]:
consumption = consumption.set_index('index', drop=True)
consumption.index.name = None

In [62]:
# convert column names to int if possible (isdigit)
consumption.columns = [int(i) if i.isdigit() else i for i in consumption.columns]
consumption.columns.name = None

In [63]:
consumption = consumption.map(lambda x: to_float(x))

In [64]:
consumption.head()

,Тип,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Все домашние хозяйства,Всего,2989.3,3582.9,4490.0,5353.1,6842.6,8561.8,9054.0,10513.4,11715.1,...,14153.8,15094.3,15295.4,16632.5,17319.9,18031.4,19727.7,19180.8,21154.3,23787.9
Все домашние хозяйства,Расходы на питание,1337.7,1537.6,1765.0,1984.1,2321.1,2947.1,3276.1,3714.6,4078.1,...,4694.9,5111.0,5707.8,6220.7,6250.0,6352.4,6818.3,7101.9,7675.2,8852.7
Все домашние хозяйства,Расходы на непродовольственные товары,1029.2,1245.9,1639.0,1976.7,2735.0,3368.2,3287.3,3906.5,4444.2,...,5600.3,5860.0,5365.5,5696.8,6230.6,6484.0,7054.8,6901.0,7418.0,8232.7
Все домашние хозяйства,Расходы на алкогольные напитки,60.5,70.1,81.7,98.2,114.3,133.8,146.3,172.1,193.6,...,233.2,257.8,263.4,284.4,279.6,284.8,313.0,337.7,320.7,379.0
Все домашние хозяйства,Расходы на оплату услуг,545.1,711.6,994.6,1283.2,1660.5,2098.1,2329.2,2705.1,2984.5,...,3606.2,3849.0,3885.3,4388.7,4520.5,4871.6,5490.6,4785.4,5686.4,6265.3


##### <font color='#AF4035'> Export Dataset to Excel

In [65]:
saveit_excel(
    data=consumption,
    filename='Домохозяйства',
    path=path_excel_csv,
    sheet='Структура потребления'
)

'Структура потребления' sheet created in file 'Домохозяйства.xlsx'


### 7.3. Располагаемые ресурсы домашних хозяйств в зависимости от места проживания по субъектам Российской Федерации

*Источник: https://rosstat.gov.ru/folder/13397*  
*Единицы измерения: рубли*

*Примечания:*  

In [66]:
resources_link = path_main + \
    ('/files/04-Population/02-Standard-of-living/05-Households/'
     'indikator_1-4.xlsx')

In [67]:
resources_link = 'https://rosstat.gov.ru/storage/mediabank/INdikator_1-4.xlsx'

In [68]:
resources_raw = pd.read_excel(resources_link, sheet_name='2014')
resources = transform_resources(
    resources_raw, 2014, FD_partial_names_list, federal_districts_names_list, 2)

/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83019/3887958554.py:2451: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(replace_dict_full)


In [69]:
for i in range(2015,2023):
    if i in [2016, 2017, 2018, 2019, 2020, 2021]:
        sl = None
    else:
        sl = 2
    df = pd.read_excel(resources_link, sheet_name=str(i))
    df = transform_resources(
        df, i, FD_partial_names_list, federal_districts_names_list, sl)
    resources = pd.concat([resources, df], axis=1)

/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83019/3887958554.py:2445: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(replace_dict_part, regex=True)
/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83019/3887958554.py:2451: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(replace_dict_full)
/var/folders/7_/v06wv5911_v97_5yw_ynf0w00000gn/T/ipykernel_83019/3887958554.py:2445: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To 

In [70]:
resources = resources.T.copy()

In [71]:
resources = resources.astype(float)

In [72]:
resources = to_round(resources, 1)

In [73]:
# check NaNs
resources[resources.isna().any(axis=1)]

,Российская Федерация,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,Курская область,Липецкая область,...,Республика Саха (Якутия),Забайкальский край,Камчатский край,Приморский край,Хабаровский край,Амурская область,Магаданская область,Сахалинская область,Еврейская автономная область,Чукотский автономный округ
,Всего,Всего,Всего,Всего,Всего,Всего,Всего,Всего,Всего,Всего,...,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность
2014,22890.1,21684.8,15585.3,19954.9,17756.8,19633.0,22200.0,17812.9,17765.7,18188.2,...,26258.4,16240.4,23523.4,25507.5,17885.1,24440.5,18484.9,21328.7,17191.7,17594.3
2015,23084.8,23863.3,16842.0,24508.7,17771.5,24677.2,20449.4,19553.4,18763.9,19996.4,...,27074.5,16014.8,25395.7,30469.9,19891.9,23817.7,24560.7,25700.4,19667.2,21072.8
2016,24209.8,23894.2,16591.0,23682.4,20737.8,25273.6,21799.8,19409.5,21219.3,20244.7,...,27824.8,17767.4,29963.1,21856.1,21874.7,29331.6,24665.9,31040.9,18896.7,21619.6
2017,24926.5,27841.2,16852.8,24261.1,24007.9,23535.2,25043.5,21265.5,22768.0,22271.8,...,32335.1,16907.1,25730.5,27438.7,19498.4,25254.0,25327.9,31826.5,20977.2,21635.0
2018,26917.7,27576.9,19037.8,24417.7,23647.6,24214.5,22829.8,21256.7,25937.7,25010.3,...,37545.5,16276.8,34103.9,24256.2,21514.7,26777.4,24733.0,33395.8,23260.5,28685.2
2019,28995.2,34497.1,20014.1,26212.3,24639.8,24931.4,25756.7,21450.9,24975.0,26092.8,...,37140.1,16693.6,36882.5,28552.9,22371.6,28524.1,25461.0,33471.3,26142.3,34497.9
2020,29204.5,28796.0,20724.4,26749.3,23022.4,27792.5,24393.2,22792.6,26022.1,24781.2,...,31337.4,18426.2,35366.2,27995.4,23308.8,27674.3,32233.4,36608.7,24964.2,35319.7
2021,31791.2,31709.7,23958.5,28835.1,24364.0,27900.5,28805.3,26698.8,30239.0,28648.9,...,31689.9,21028.7,42615.7,24472.5,25816.3,29084.3,39330.4,35999.4,32201.4,61573.6
2022,35066.1,30709.8,24075.7,33858.3,29701.7,31388.4,27115.8,29263.4,32612.9,30490.5,...,32319.0,25369.1,43823.9,25405.3,35040.9,38349.0,45930.1,35133.5,31342.6,58409.0


In [74]:
resources

,Российская Федерация,Белгородская область,Брянская область,Владимирская область,Воронежская область,Ивановская область,Калужская область,Костромская область,Курская область,Липецкая область,...,Республика Саха (Якутия),Забайкальский край,Камчатский край,Приморский край,Хабаровский край,Амурская область,Магаданская область,Сахалинская область,Еврейская автономная область,Чукотский автономный округ
,Всего,Всего,Всего,Всего,Всего,Всего,Всего,Всего,Всего,Всего,...,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность,Сельская местность
2014,22890.1,21684.8,15585.3,19954.9,17756.8,19633.0,22200.0,17812.9,17765.7,18188.2,...,26258.4,16240.4,23523.4,25507.5,17885.1,24440.5,18484.9,21328.7,17191.7,17594.3
2015,23084.8,23863.3,16842.0,24508.7,17771.5,24677.2,20449.4,19553.4,18763.9,19996.4,...,27074.5,16014.8,25395.7,30469.9,19891.9,23817.7,24560.7,25700.4,19667.2,21072.8
2016,24209.8,23894.2,16591.0,23682.4,20737.8,25273.6,21799.8,19409.5,21219.3,20244.7,...,27824.8,17767.4,29963.1,21856.1,21874.7,29331.6,24665.9,31040.9,18896.7,21619.6
2017,24926.5,27841.2,16852.8,24261.1,24007.9,23535.2,25043.5,21265.5,22768.0,22271.8,...,32335.1,16907.1,25730.5,27438.7,19498.4,25254.0,25327.9,31826.5,20977.2,21635.0
2018,26917.7,27576.9,19037.8,24417.7,23647.6,24214.5,22829.8,21256.7,25937.7,25010.3,...,37545.5,16276.8,34103.9,24256.2,21514.7,26777.4,24733.0,33395.8,23260.5,28685.2
2019,28995.2,34497.1,20014.1,26212.3,24639.8,24931.4,25756.7,21450.9,24975.0,26092.8,...,37140.1,16693.6,36882.5,28552.9,22371.6,28524.1,25461.0,33471.3,26142.3,34497.9
2020,29204.5,28796.0,20724.4,26749.3,23022.4,27792.5,24393.2,22792.6,26022.1,24781.2,...,31337.4,18426.2,35366.2,27995.4,23308.8,27674.3,32233.4,36608.7,24964.2,35319.7
2021,31791.2,31709.7,23958.5,28835.1,24364.0,27900.5,28805.3,26698.8,30239.0,28648.9,...,31689.9,21028.7,42615.7,24472.5,25816.3,29084.3,39330.4,35999.4,32201.4,61573.6
2022,35066.1,30709.8,24075.7,33858.3,29701.7,31388.4,27115.8,29263.4,32612.9,30490.5,...,32319.0,25369.1,43823.9,25405.3,35040.9,38349.0,45930.1,35133.5,31342.6,58409.0


In [75]:
get_data_two_level(resources, level0=['Республика Татарстан'])

Республика Татарстан                                       
                    Всего Городская местность Сельская местность
2014              27076.3             30306.1            16797.8
2015              25797.4             28026.3            18638.8
2016              25343.4             27901.4            17057.9
2017              27184.1             29558.6            19409.5
2018              33374.3             37625.0            19307.9
2019              28138.4             30404.9            20588.4
2020              30706.4             32726.6            23972.7
2021              32746.2             35077.0            25307.0
2022              35341.8             37228.2            29079.8

##### <font color='#AF4035'> Export Dataset to Excel

In [76]:
saveit_excel(
    data=resources,
    filename='Домохозяйства',
    path=path_excel_csv,
    sheet='Ресурсы по регионам'
)

'Ресурсы по регионам' sheet created in file 'Домохозяйства.xlsx'


### Save Dataset

In [77]:
economics_data['households_income'] = income

In [78]:
economics_data['households_consumption'] = consumption

In [79]:
economics_data['households_resources'] = resources

In [80]:
saveit(
    file=economics_data,
    name='economics_data',
    path=path_files
)

File 'economics_data.pkl' saved


### Execution time

In [81]:
stopwatch_stop(start)

Execution time: 0:00:20
